# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [21]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from tqdm.notebook import tqdm
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import joblib
import warnings

warnings.filterwarnings('ignore')

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).


In [3]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['timestamp'] = pd.to_datetime(X['timestamp'])
        X['hour'] = X['timestamp'].dt.hour
        X['weekday'] = X['timestamp'].dt.weekday
        X.drop('timestamp', axis=1, inplace=True)
        return X

In [4]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, target_col):
        self.target_col = target_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_cat = X.select_dtypes(include=['object'])
        if self.target_col in X_cat.columns:
            X_cat.drop(self.target_col, axis=1, inplace=True)
        enc = OneHotEncoder(sparse_output=False)
        X_encoded = enc.fit_transform(X_cat)
        X.drop(X_cat.columns, axis=1, inplace=True)
        X = pd.concat([X, pd.DataFrame(X_encoded, columns=enc.get_feature_names_out())], axis=1)
        return X

In [5]:
class TrainValidationTest(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=21, stratify=y_train)
        return X_train, X_valid, X_test, y_train, y_valid, y_test

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907

Classifier with best validation set accuracy: Random Forest
```

In [19]:
class ModelSelection:
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grid_dict = grid_dict
        self.best_models = []

    def choose(self, X_train, y_train, X_valid, y_valid):
        for grid_search in tqdm(self.grids):
            model_name = self.grid_dict[self.grids.index(grid_search)]
            print(f"Estimator: {model_name}")
            grid_search.fit(X_train, y_train)
            valid_score = grid_search.score(X_valid, y_valid)
            best_params = grid_search.best_params_
            self.best_models.append((model_name, best_params, valid_score))
            print("Best params:", best_params)
            print("Best training accuracy:", grid_search.score(X_train, y_train))
            print("Validation set accuracy score for best params:", valid_score)
            print()


        best_model = max(self.best_models, key=lambda x: x[2])
        return best_model[0]

    def best_results(self):
        df = pd.DataFrame(self.best_models, columns=['model', 'params', 'valid_score'])
        return df

## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [25]:
class Finalize:
    def __init__(self, estimator):
        self.estimator = estimator

    def final_score(self, X_train, y_train, X_test, y_test):
        self.estimator.fit(X_train, y_train)
        y_pred = self.estimator.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy of the final model is {accuracy}")
        return accuracy

    def save_model(self, path):
        with open(path, 'wb') as file:
            pickle.dump(self.estimator, file)
        print(f"Model was successfully saved to {path}")

## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

In [2]:
df = pd.read_csv('/content/checker_submits.csv', parse_dates=['timestamp'])
df.head()


,uid,labname,numTrials,timestamp
0,user_4,project1,1,2020-04-17 05:19:02.744528
1,user_4,project1,2,2020-04-17 05:22:45.549397
2,user_4,project1,3,2020-04-17 05:34:24.422370
3,user_4,project1,4,2020-04-17 05:43:27.773992
4,user_4,project1,5,2020-04-17 05:46:32.275104


In [6]:
pipeline = Pipeline([
    ('feature_extractor', FeatureExtractor()),
    ('one_hot_encoder', MyOneHotEncoder(target_col='weekday'))
])

df_prepared = pipeline.fit_transform(df)

In [7]:
X = df_prepared.drop('weekday', axis=1)
y = df_prepared[['weekday']]
X_train, X_valid, X_test, y_train, y_valid, y_test = TrainValidationTest().transform(X, y)

In [22]:
svm_params = [{'kernel':['linear', 'rbf'],
               'C':[0.01, 1.5, 5, 10],
               'gamma': ['scale', 'auto'],
               'class_weight':('balanced', None),
               'random_state':[21],
               'probability':[True]}]
tree_params = {
    'random_state':[21],
    'max_depth': range(1, 30),
    'class_weight': ['balanced', None],
    'criterion': ['entropy', 'gini']
}
rf_params = {
    'random_state':[21],
    'n_estimators': [5, 50, 100],
    'max_depth': range(1, 30),
    'class_weight': ['balanced', None],
    'criterion': ['entropy', 'gini']
}
gs_svm = GridSearchCV(estimator=SVC(), param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_tree = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=tree_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_params, scoring='accuracy', cv=2, n_jobs=-1)

grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: 'SVM', 1: 'Decision Tree', 2: 'Random Forest'}

model_selection = ModelSelection(grids, grid_dict)
best_model_name = model_selection.choose(X_train, y_train, X_valid, y_valid)
print(f"Лучшая модель: {best_model_name}")


  0%|          | 0/3 [00:00<?, ?it/s]

Estimator: SVM
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.9525222551928784
Validation set accuracy score for best params: 0.8545994065281899

Estimator: Decision Tree
Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 17, 'random_state': 21}
Best training accuracy: 0.9851632047477745
Validation set accuracy score for best params: 0.8605341246290801

Estimator: Random Forest
Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 22, 'n_estimators': 100, 'random_state': 21}
Best training accuracy: 1.0
Validation set accuracy score for best params: 0.884272997032641

Лучшая модель: Random Forest


In [23]:
model_selection.best_results()

,model,params,valid_score
0,SVM,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.854599
1,Decision Tree,"{'class_weight': None, 'criterion': 'gini', 'm...",0.860534
2,Random Forest,"{'class_weight': None, 'criterion': 'gini', 'm...",0.884273


In [26]:
best_params = {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
estimator = RandomForestClassifier(**best_params)

final = Finalize(estimator)

final.final_score(X_train, y_train, X_test, y_test)

final.save_model('finalized_model.sav')

Accuracy of the final model is 0.9023668639053254
Model was successfully saved to finalized_model.sav
